In [ ]:
%matplotlib inline

import seaborn as sns
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import igraph as ig
import networkx as nx
import cdlib


En esta ocasión construiremos los conceptos básicos de la autocorrelación espacial utilizando herramientas básicas. Específicamente construiremos la noción de vecinos, la matriz de pesos, lag espacial y el índice de Moran, mas adelante veremos la implementación en pysal

In [ ]:
zona_diss=gpd.read_file('datosZonas_Stgo_C2017.gpkg')

In [ ]:
zona_diss

Usarémos la vecindad de reina, es decir que al menos un punto toque entre dos unidades espaciales. Notaremos que la noción de vecinos es un grafo y su matriz es el equivalente a una matriz de incidencia.

In [ ]:
%%time
nbrs=[zona_diss.geometry.intersects(j) for j in zona_diss.geometry]
nbrs

In [ ]:
%%time
list_of_nbrs=[list(filter(lambda i: nbrs[j][i]==True, range(len(nbrs[j])))) for j in range(len(nbrs))]
list_of_nbrs

El siguiente codigo construye el data frame desde el cual construiremos el grafo

In [ ]:
%%time
nbrs_graph_df=pd.DataFrame(columns=['self', 'other'])
selfs=[]
others=[]

for i in range(len(list_of_nbrs)):
    self=i
    for j in list_of_nbrs[i]:
        other=j
        if self != other:
            selfs.append(self)
            others.append(other)

    

In [ ]:
nbrs_graph_df['self']=selfs
nbrs_graph_df['other']=others

In [ ]:
nbrs_graph_df

Para el manejo de grafos usaremos las librerias networkx I graph, específicamente lo utilizaremos para visualizar la estructura de grafo y construir la matriz de incidencia (vecinos)

In [ ]:
G=nx.Graph()
for i in range(len(nbrs_graph_df)):
    G.add_edge(nbrs_graph_df['self'][i], nbrs_graph_df['other'][i])

In [ ]:
zona_diss['x']= zona_diss['geometry'].centroid.x
zona_diss['y']= zona_diss['geometry'].centroid.y

In [ ]:
pos=dict()
for i in range(len(zona_diss)):
    key=i
    value=(zona_diss[i:i+1]['x'].values[0], zona_diss[i:i+1]['y'].values[0])
    pos[key]=value

In [ ]:
nx.set_node_attributes(G, pos, 'coord')

In [ ]:
zona_diss

In [ ]:
zona_diss.crs

In [ ]:
fig=plt.figure(figsize=(15,25))

ax=fig.add_subplot(1,1,1)
nx.draw_networkx(G, pos, node_size=10, with_labels=False, ax=ax, width =1)
zona_diss.boundary.plot(ax=ax)


In [ ]:
Gi=cdlib.utils.convert_graph_formats(G, ig.Graph, directed=None)

In [ ]:
nbrs_adj=Gi.get_adjacency_sparse()

Una vez tenemos la matriz de adyacencia (asumiendo que no se construyó directamente una matriz de pesos o un grafo con peso), podemos construir los pesos a traves de la matriz de adyacencia rápidamente (usaremos peso por normalización de fila)

In [ ]:
nbrs_adj.sum(axis=1)

In [ ]:
weight_matrix=nbrs_adj/nbrs_adj.sum(axis=1)

In [ ]:
mean=np.mean(zona_diss['INMIGRANTE'])
std=np.std(zona_diss['INMIGRANTE'])
normed=(zona_diss['INMIGRANTE']-mean)/std
zona_diss['normed_inm']=normed

Una vez se tiene la matriz de pesos, el lag espacial es una simple multiplicación entre la matriz de pesos y el vector del atributo de interés, es decir, para cada punto la suma ponderada de vecinos

In [ ]:
splag=weight_matrix.dot(normed)
splag

In [ ]:
mask=weight_matrix[-2]!=0
weight_matrix[-2]

In [ ]:
zona_diss[[mask[0,i] for i in range(mask.shape[1])]]['normed_inm']

In [ ]:
sum(zona_diss[[mask[0,i] for i in range(mask.shape[1])]]['normed_inm'])*0.14285714

In [ ]:
zona_diss['splag']=[splag[0,i] for i in range(splag.shape[1])]

El Scatterplot de Moran permite visualizar la relación entre el lag espacial y el atributo.

In [ ]:
f, ax = plt.subplots(1, figsize=(9, 9))

plt.plot(zona_diss['normed_inm'], zona_diss['splag'], '.', color='firebrick')

 # dashed vert at mean of the last year's PCI
plt.vlines(zona_diss['normed_inm'].mean(), zona_diss['splag'].min(), zona_diss['splag'].max(), linestyle='--')
 # dashed horizontal at mean of lagged PCI
plt.hlines(zona_diss['splag'].mean(), zona_diss['normed_inm'].min(), zona_diss['normed_inm'].max(), linestyle='--')

# red line of best fit using global I as slope
#plt.plot(HR90, a + b*HR90, 'r')
plt.title('Moran Scatterplot Fruna')
plt.ylabel('Spatial Lag of INMIGRANTE')
plt.xlabel('INMIGRANTE')
plt.show()

El Scatterplot de Moran ademas contiene la I global de moran que caracteriza la autocorrelación espacial, esta corresponde a la pendiente del modelo lineal entre el atributo y el lag

In [ ]:
from sklearn.linear_model import LinearRegression as lr

In [ ]:
reg=lr()

In [ ]:
reg.fit(zona_diss['normed_inm'].values.reshape(-1, 1), zona_diss['splag'].values.reshape(-1, 1))

In [ ]:
reg.coef_

In [ ]:
reg.intercept_

In [ ]:
f, ax = plt.subplots(1, figsize=(9, 9))

plt.plot(zona_diss['normed_inm'], zona_diss['splag'], '.', color='firebrick')

 # dashed vert at mean of the last year's PCI
plt.vlines(zona_diss['normed_inm'].mean(), zona_diss['splag'].min(), zona_diss['splag'].max(), linestyle='--')
 # dashed horizontal at mean of lagged PCI
plt.hlines(zona_diss['splag'].mean(), zona_diss['normed_inm'].min(), zona_diss['normed_inm'].max(), linestyle='--')
plt.plot([-1,0,1,2,3,4,5,6, 7, 8], reg.predict(np.array([-1,0,1,2,3,4,5,6, 7, 8]).reshape(-1, 1)))
# red line of best fit using global I as slope
#plt.plot(HR90, a + b*HR90, 'r')
plt.title('Moran Scatterplot')
plt.ylabel('Spatial Lag of INMIGRANTE')
plt.xlabel('INMIGRANTE')
plt.show()